# Multiple linear regression

This exercise is useful to see:
- multiple linear regression
- scatterplot matrix
- correlation matrix
- collinearity problem
- the residuals, the studentized residuals and the leverage points
- interaction effects
- non-linear transformations of the predictors

Import all the packages that we need.

In [ ]:
import statsmodels.api as sm
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from pandas.plotting import scatter_matrix
from mpl_toolkits.mplot3d import Axes3D

from statsmodels.stats.outliers_influence import OLSInfluence
from statsmodels.stats.outliers_influence import variance_inflation_factor

Read the data from Auto.csv and remove all rows, that are not complete.

In [ ]:
auto_df = pd.read_csv('Data/Auto.csv', na_values='?')
auto_df = auto_df.dropna()
auto_df.head()

a) Produce a scatterplot matrix which includes all the variables in the dataset

In [ ]:

scatter_matrix(auto_df, alpha=0.5, figsize=(20,20))

b) Compute the matrix of correlations between the variables. 

In [ ]:
corr = auto_df.corr(numeric_only=True)
corr.style.background_gradient(cmap='coolwarm')


c) Perform a multiple linear regression with mpg as the response and all other variables (except 'name') as the predictors. Print the results and comment on outputs.

In [ ]:
X = sm.add_constant(auto_df.iloc[:,1:-1]) # add all values except mpg and names
y = auto_df.mpg

model = sm.OLS(y,X)
estimate = model.fit()
print(estimate.summary())

d) Compute the variance inflation factors and comment the output.

In [ ]:
VIFs = [(predictor, variance_inflation_factor(X.values, _)) for _, predictor in enumerate(list(X))]
for tup in VIFs:
    print('{:20}'.format(tup[0]), '{:.3f}'.format(tup[1]))

VIF = 1 -> Absence of collinearity;
VIF > 5 -> Multi-Collinearity
VIF > 10 -> Strong Multi-Collinearity

e) Produce diagnostic plots of the linear regression fit. Comment on any problems you see with the fit. Do the residual plot suggests any unusually large outliers? Does the leverage plot identify any observations with unusually high leverage?

In [ ]:
fitted_values = estimate.fittedvalues
residuals = estimate.resid.values
studentized_residuals = OLSInfluence(estimate).resid_studentized_internal
leverages = OLSInfluence(estimate).influence

fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(16,4))
ax1.scatter(fitted_values, residuals, facecolor='none', edgecolor='b')
ax1.set_xlabel('fitted values')
ax1.set_ylabel('residuals')

ax2.scatter(fitted_values, studentized_residuals, facecolor='none', edgecolor='b')
ax2.set_xlabel('fitted values')
ax2.set_ylabel('studentized residuals')

ax3.scatter(leverages, studentized_residuals, facecolor='none', edgecolor='b')
ax3.set_xlabel('leverages')
ax3.set_ylabel('studentized_residuals')

Residuals over fitted values shows a heavy U-shaped graph. This indicates a poor fit for a simple linear regression in the higher fitted values.

Studentized residuals over fitted values shows a U-shaped graph and heavy outliers with a >3 studentized residual value. This indicates an underfit.

Studentized residuals over leverages shows many values for leverages over (p+1)/n = (8+1)/392 = 0.02 here. This indicates a poor fit especially for outliers (duh) -> Better to use some quadratic function for fitting.

f) Fit linear regression models with interaction effects. Do any interactions appear to be statistically significant? Use the example:
$\text{mgp} = \beta_0 + \beta_1\cdot\text{weight} + \beta_2\cdot\text{year} + \beta_3\cdot (\text{weight}\cdot \text{year})$ 

In [ ]:
auto_df['weight*year'] = auto_df.weight*auto_df.year
X_interaction = sm.add_constant(auto_df[['weight', 'year', 'weight*year']])
y = auto_df.mpg

model = sm.OLS(y,X_interaction)
estimate = model.fit()
print(estimate.summary())


g) Try a few different transformation of the variables, such as:

i) Add a $\text{weight}^2$ variable variable to the model of task f).

In [ ]:
auto_df['weight^2'] = auto_df.weight**2
X_interaction = sm.add_constant(auto_df[['weight', 'weight^2', 'year', 'weight*year']])
y = auto_df.mpg

model = sm.OLS(y,X_interaction)
estimate = model.fit()
print(estimate.summary())

# Obtain the residuals, studentized residuals and the leverages
fitted_values = estimate.fittedvalues
residuals = estimate.resid.values
studentized_residuals = OLSInfluence(estimate).resid_studentized_internal
leverages = OLSInfluence(estimate).influence

# Plot
fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(16,4))
ax1.scatter(fitted_values, residuals, facecolor='none', edgecolor='b')
ax1.set_xlabel('fitted values')
ax1.set_ylabel('residuals')

# Studentized Residuals
ax2.scatter(fitted_values, studentized_residuals, facecolor='none', edgecolor='b')
ax2.set_xlabel('fitted values')
ax2.set_ylabel('studentized residuals')

# Leverages
ax3.scatter(leverages, studentized_residuals, facecolor='none', edgecolor='b')
ax3.set_xlabel('leverages')
ax3.set_ylabel('studentized_residuals')

ii) Add a $\text{weight}^{\frac{1}{2}}$ variable to the model of task f).

In [ ]:
auto_df['weight^0.5'] = auto_df.weight**0.5
X_interaction = sm.add_constant(auto_df[['weight', 'year', 'weight^0.5']])

model = sm.OLS(y,X_interaction)
estimate = model.fit()
print(estimate.summary())

fitted_values = estimate.fittedvalues
residuals = estimate.resid.values
studentized_residuals = OLSInfluence(estimate).resid_studentized_internal
leverages = OLSInfluence(estimate).influence

fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(16,4))
ax1.scatter(fitted_values, residuals, facecolor='none', edgecolor='b')
ax1.set_xlabel('fitted values')
ax1.set_ylabel('residuals')

ax2.scatter(fitted_values, studentized_residuals, facecolor='none', edgecolor='b')
ax2.set_xlabel('fitted values')
ax2.set_ylabel('studentized residuals')

ax3.scatter(leverages, studentized_residuals, facecolor='none', edgecolor='b')
ax3.set_xlabel('leverages')
ax3.set_ylabel('studentized_residuals')

iii) Add a $\log(\text{weight})$ variable to the model of task f).

In [ ]:
auto_df['log(weight)'] = np.log(auto_df.weight)
X_interaction = sm.add_constant(auto_df[['weight', 'year', 'log(weight)']])

model = sm.OLS(y,X_interaction)
estimate = model.fit()
print(estimate.summary())

fitted_values = estimate.fittedvalues
residuals = estimate.resid.values
studentized_residuals = OLSInfluence(estimate).resid_studentized_internal
leverages = OLSInfluence(estimate).influence

fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(16,4))
ax1.scatter(fitted_values, residuals, facecolor='none', edgecolor='b')
ax1.set_xlabel('fitted values')
ax1.set_ylabel('residuals')

ax2.scatter(fitted_values, studentized_residuals, facecolor='none', edgecolor='b')
ax2.set_xlabel('fitted values')
ax2.set_ylabel('studentized residuals')

ax3.scatter(leverages, studentized_residuals, facecolor='none', edgecolor='b')
ax3.set_xlabel('leverages')
ax3.set_ylabel('studentized_residuals')